In [6]:
import torch
import numpy as np
import pandas as pd

eval_path = "../facebase/results/Adience_256x256_resnet50_imagenet_noisy_dldl_v2_nr2_smcarl_v3/split0/evaluation.pt"
ev = torch.load(eval_path)

# Extract necessary data
id_list = ev['id']
folder_list = ev['folder']
true_label = np.array(ev['true_label']['age'])
# true_label = df[3].to_numpy()
predicted_label = np.array(ev['predicted_label']['age'])

# Filter test indices where folder == 2
test_indices = [i for i in range(len(id_list)) if folder_list[i] == 2]

y_true = true_label[test_indices]
y_pred = predicted_label[test_indices]

In [7]:
def rank_acc_mae(y_pre, y, num_class):
    df = pd.DataFrame({'pre':y_pre.cpu().numpy()})
    rank_pre_num_list = torch.zeros(num_class,device=y_pre.device)
    rank_acc_list = torch.zeros(num_class,device=y_pre.device)
    rank_mae_list = torch.zeros(num_class, device=y_pre.device)
    for rank in range(num_class):
        # pre num
        rank_pre_num = sum(y_pre==rank)
        rank_pre_num_list[rank]= rank_pre_num
        # pre acc&mae
        y_rank_pre = y_pre[y == rank]
        if len(y_rank_pre)==0:
            y_rank_acc=0
            y_rank_mae=np.inf
        else:
            y_rank_acc = sum(y_rank_pre == rank)/len(y_rank_pre)
            y_rank_mae = torch.sum(torch.abs(y_rank_pre-rank))/len(y_rank_pre)
        rank_acc_list[rank] = y_rank_acc
        rank_mae_list[rank] = y_rank_mae
    return rank_acc_list, rank_mae_list, rank_pre_num_list


In [ ]:
rank_acc_mae(y_pre=torch.tensor(y_pred), y=torch.tensor(y_true), num_class=8)

(tensor([0.6398, 0.7842, 0.3382, 0.3216, 0.6714, 0.3446, 0.2490, 0.3385]),
 tensor([0.3892, 0.2825, 0.9265, 0.7269, 0.3551, 0.7490, 1.0954, 0.9494]),
 tensor([ 332.,  636.,  236.,  373., 1157.,  556.,  285.,  104.]))

In [10]:
acc, mae, n = rank_acc_mae(y_pre=torch.tensor(y_pred), y=torch.tensor(y_true), num_class=8)
torch.mean(acc), torch.mean(mae)

(tensor(0.4609), tensor(0.6842))

In [12]:
#MAE
def MAE(Y,Y_pre):
    from sklearn.metrics import mean_absolute_error
    return mean_absolute_error(Y,Y_pre)
#RMSE
def RMSE(Y,Y_pre):
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(Y,Y_pre))
#ACC
def ACC(Y,Y_pre):
    from sklearn.metrics import accuracy_score
    return accuracy_score(Y,Y_pre)

In [13]:
MAE(y_true, y_pred)

0.5678173416689318